## 02. Train the model. Gets kmeans dictionary

In [ ]:
sc.version
val conf = sc.hadoopConfiguration
val fs = org.apache.hadoop.fs.FileSystem.get(conf)
val hdfshome = fs.getHomeDirectory
println(sc.version)
println(hdfshome)
val files = fs.listFiles(hdfshome, false)

In [5]:
val rootDir = "/user/fsainz/data/out"
val files = fs.listFiles(new org.apache.hadoop.fs.Path(rootDir), false)


In [6]:
//fs.exists(new org.apache.hadoop.fs.Path("hdfs://spark-clu-m/user/fsainz/data/in/a02.dat"))
fs.exists(new org.apache.hadoop.fs.Path(rootDir))

true

## Split in windows


In [1]:
import java.io._                                                        
import scala.collection.mutable.ListBuffer   //bE CAREFULL With java imports, better before scala.

val WINDOW = 120;
val STEP = 4;
val TRsamples = 200000;
val scale = 1/200.0;
//val inFileName = "a02.dat"     //apnea
val inFileName = "105s1.dat"   //mit-arritmia

val inputPath =  "/home/fsainz/data/" //local filesystem
val fileName = inputPath + inFileName 
val hdfsOutPath = "/user/fsainz/data/out/"
val outFileName = hdfsOutPath + "dict-" + inFileName.split('.')(0) + "_W" + WINDOW +".LEmodel"


println("InFile: " + fileName)
println("Ouput Model: " + outFileName)

InFile: /home/fsainz/data/105s1.dat
Ouput Model: /user/fsainz/data/out/dict-105s1_W120.LEmodel


In [2]:


// Now start reading local file   TRaining file is part InFile training and run

var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)


// val nSamples =  (inFile.length() / 2)   // We are going to read short: 2 bytes..


val nWindows = TRsamples / WINDOW
println("nWindows = " + nWindows.toString)

val buffer = new Array[Double](WINDOW)
var windows = scala.collection.mutable.ListBuffer.empty[Array[Double]]

var pos = 0
var available = 0
var dataWindow = new Array[Double](WINDOW)

for (  w  <- 1 to nWindows.toInt){
    // new object is neeed to append at the end!!
  dataWindow = new Array[Double](WINDOW) 
  var newWin:Boolean = false
  while( ! newWin) {
      
    // .dat files are little endian 
    val value = java.lang.Short.reverseBytes(input.readShort())
    //print ("%x ".format(value), value)
    buffer( pos ) = (value.toDouble * scale) 
    available += 1
    pos +=1
    if ( pos == WINDOW )  // buffer ends
      pos = 0  
    if (available == WINDOW) {    
      var j =0
      for( i <- pos until WINDOW){
         dataWindow(j) = buffer(i)
         j +=1
      }
      for( i <- 0 until pos ){
         dataWindow(j) = buffer(i)
         j +=1
      }
      //dataWindow.foreach(print)
      //println("|")
      newWin = true
      available = WINDOW-STEP   // ready for read windows displacement offset 
    }
  }
  windows.append(dataWindow  )  
}
input.close()

nWindows = 1666


In [3]:
nWindows*WINDOW*2

399840

## Defining windowing function

In [4]:

var windowingF  =  new Array[Double](WINDOW)   //differs of Array[Double](WINDOW) uajj
for ( i <- 0 until WINDOW) {
    val y = Math.sin(Math.PI * i / (WINDOW - 1.0));
          windowingF(i)=  y * y
 }


In [5]:
val ECGRdd = sc.parallelize(windows)


In [6]:
// spark 2.1 mllib
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.regression.LabeledPoint 

In [7]:
val ECGwnRdd = ECGRdd.map( x => ((x,windowingF).zipped.map(_*_))).map(x =>( x, Vectors.norm( Vectors.dense(x),2) )).map(x => Vectors.dense(x._1.map( c => c/ x._2)) ) 
ECGwnRdd.count
ECGwnRdd.first

[0.0,1.012314263880639E-4,4.0464355541035393E-4,9.093907230442163E-4,0.0016140661084035513,0.0025167056548009467,0.003614793543925933,0.004905269207881844,0.006377707478939039,0.00807429448699254,0.009913595787027066,0.012013206129932157,0.014280366134827152,0.0167322054182821,0.019323571305776936,0.022101607320991853,0.025042562239781404,0.028169027604718466,0.03145115177018974,0.03491762007786862,0.038452935898706575,0.04215667424037047,0.045656810694352463,0.04952196991069394,0.05348414800125847,0.05753337213625115,0.061534544920459504,0.06545285705650494,0.06932339379951591,0.07334646968019955,0.07745668860932527,0.08140666562467924,0.08533442188010683,0.08922883342189634,0.09279395654519877,0.09667583177614536,0.100808408734863,0.104255...

In [8]:
    ECGwnRdd.cache()  // important to kmeans time
    val numIterations = 20// we want to assure a global mininum with 20 it seem not to be reached
    val numClusters = 400
    val clusters = KMeans.train(ECGwnRdd, numClusters, numIterations)

In [9]:
clusters

org.apache.spark.mllib.clustering.KMeansModel@570a3627

In [10]:
clusters.save(sc, outFileName)
println(outFileName + " SAVED")

/user/fsainz/data/out/dict-105s1_W120.LEmodel SAVED
